In [1]:
from square.client import Client
import os
import pandas as pd

# import pandas as pd

# import os
import pickle

In [2]:
client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [3]:
orders_api = client.orders
customers_api = client.customers

In [4]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
        'sort': {
            'sort_field': 'CREATED_AT',
            'sort_order': 'DESC'
        },
    'query': {
        'filter': {
            
            'date_time_filter': {
                'created_at': {
                    'start_at': '2023-08-30T00:00:00+00:00',
                }
            }
        },
        'sort': {
            'sort_field': 'CREATED_AT',
            'sort_order': 'ASC'
        }
    },
   }

In [5]:
s_orders = orders_api.search_orders(body).body['orders']
len(s_orders)

70

In [6]:
pd.DataFrame(s_orders)['line_items'][0]

[{'uid': '2F5E185E-CEBD-4853-92EF-FEE92563BD54',
  'catalog_object_id': 'OKAX666RRB7XKNM6BJTRM6IM',
  'catalog_version': 1691427378695,
  'quantity': '1',
  'name': 'Match Sub',
  'variation_name': 'Full Game',
  'base_price_money': {'amount': 1000, 'currency': 'GBP'},
  'gross_sales_money': {'amount': 1000, 'currency': 'GBP'},
  'total_tax_money': {'amount': 0, 'currency': 'GBP'},
  'total_discount_money': {'amount': 0, 'currency': 'GBP'},
  'total_money': {'amount': 1000, 'currency': 'GBP'},
  'variation_total_price_money': {'amount': 1000, 'currency': 'GBP'},
  'item_type': 'ITEM',
  'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}]

In [7]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

In [8]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'created_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)

    for item in order['line_items']:
        line_dict = new_order.copy()
        
        line_dict['variation_name']=get_value(item,'variation_name')
        line_dict['item_name']=get_value(item,'name')
        line_dict['note']=get_value(item,'note')
        line_dict['total_money']=item['total_money']['amount']/100
        orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

0 / 70
1 / 70
2 / 70
3 / 70
4 / 70
5 / 70
6 / 70
7 / 70
8 / 70
9 / 70
10 / 70
11 / 70
12 / 70
13 / 70
14 / 70
15 / 70
16 / 70
17 / 70
18 / 70
19 / 70
20 / 70
21 / 70
22 / 70
23 / 70
24 / 70
25 / 70
26 / 70
27 / 70
28 / 70
29 / 70
30 / 70
31 / 70
32 / 70
33 / 70
34 / 70
35 / 70
36 / 70
37 / 70
38 / 70
39 / 70
40 / 70
41 / 70
42 / 70
43 / 70
44 / 70
45 / 70
46 / 70
47 / 70
48 / 70
49 / 70
50 / 70
51 / 70
52 / 70
53 / 70
54 / 70
55 / 70
56 / 70
57 / 70
58 / 70
59 / 70
60 / 70
61 / 70
62 / 70
63 / 70
64 / 70
65 / 70
66 / 70
67 / 70
68 / 70
69 / 70


In [9]:
orders_df['customer_name']= orders_df['first_name'] + ' ' + orders_df['last_name']
match_fees_export = orders_df[(orders_df['customer_id'].notnull()) | (orders_df['note'].notnull())][['id', 'created_at', 'variation_name','item_name', 'customer_name', 'total_money', 'note']]


match_fees_export = match_fees_export[(match_fees_export['item_name'].fillna('').str.lower().str.contains('sub'))|(match_fees_export['item_name'].fillna('').str.lower().str.contains('training'))]

In [10]:
match_fees_export

,id,created_at,variation_name,item_name,customer_name,total_money,note
0,r1Ooztszirds0M6KNslIHQreV,2023-08-30T06:17:38Z,Full Game,Match Sub,Andreas Andreou,10.0,None
1,HbKKWu2JSwobf82Nw0Cn2RveV,2023-08-30T07:32:04Z,Full Game,Match Sub,Peter Green,10.0,None
2,9whNP9hXRQJLgww6n2RjgSxeV,2023-08-30T08:28:20Z,U23 Full Game,Match Sub,Lawrence Asante,10.0,None
3,Nh5wAl7vLJT3ondqfYhrPY99KaIZY,2023-08-30T08:35:55.282Z,None,Match Subs,Jamie Arkell,10.0,None
4,jXFUOIut2ZdbevwftSyVV58eV,2023-08-30T08:52:49Z,Full Game,A Match Sub,Anitej Rao,10.0,Vs London Hibs 05/08/2023
...,...,...,...,...,...,...,...
67,fhh1rJ0zjm90I9QyWpuXu0yeV,2023-08-31T19:06:49Z,Regular,Training Sub,Ryan Cochrane,5.0,None
68,5c0lgJ2UXaVCKpiYz6u5zx0eV,2023-08-31T19:07:12Z,Regular,Training Sub,Andres Ahlgren,5.0,None
69,BECbWAq8U6HpGfNaUj1T4IreV,2023-08-31T20:04:26Z,Regular,Training Sub,Christian King,5.0,None
70,BECbWAq8U6HpGfNaUj1T4IreV,2023-08-31T20:04:26Z,Full Game,Match Sub,Christian King,10.0,None


In [33]:


exist = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)
    

In [34]:
exist = get_spread_sheet()

In [35]:
exist

,ID,Date,Cat,Item,Customer,Price,Notes
0,73EB3FDF,Aggregate,Match Subs,Match Subs,Ade Lawal,10,None
1,EAC2E5C7,Aggregate,Match Subs,Match Subs,Aidan Byrne,10,None
2,FBDA8A4F,Aggregate,Match Subs,Match Subs,Alex Clark,10,None
3,DBD405F9,Aggregate,Match Subs,Match Subs,Alex Dockerill,20,None
4,15E18EA1,Aggregate,Match Subs,Match Subs,Alex Mansfield,10,None
...,...,...,...,...,...,...,...
374,T6kkwkmPg8DXm2amkK4319OIK48YY,8/30/2023 18:45:10,None,Match Subs,Tali Kiwanuka,10.00,None
375,dhzXWkeVIHJc8vfsSKedjJyCMrFZY,8/30/2023 19:22:23,None,Training,PAUL BALL,5.00,None
376,d47UVoXAWjwNXnJorWM1E3meV,8/31/2023 14:25:32,Match Subs,Match Sub,Mike Moon,10.00,None
377,1s3e8C5ZWyz96EqGpce2F38eV,8/31/2023 11:27:26,Match Subs,Match Sub,Leigh Cremin,10.00,None


In [36]:
match_fees_export.columns = exist.columns

In [39]:
mg = pd.merge(left=match_fees_export, right=exist, how='left', on='ID', suffixes=['', '_old'])
not_in = mg[mg['Date_old'].isnull()][exist.columns]

In [46]:
not_in.fillna('NA', inplace=True)

In [47]:
import pandas as pd





credentials = Credentials.from_service_account_file('ots_service.json', scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# open a google sheet
gs = gc.open_by_key(SAMPLE_SPREADSHEET_ID_input)
# select a work sheet from its name
worksheet1 = gs.worksheet('Square Subs')

In [48]:
df_values = not_in.values.tolist()
gs.values_append('Square Subs', {'valueInputOption': 'RAW'}, {'values': df_values})

{'spreadsheetId': '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8',
 'tableRange': "'Square Subs'!A1:G380",
 'updates': {'spreadsheetId': '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8',
  'updatedRange': "'Square Subs'!A381:G427",
  'updatedRows': 47,
  'updatedColumns': 7,
  'updatedCells': 329}}